## More Advanced Python

There are several advanced Python notions.

### Decorators

**Decorator:** is a function that takes another function as input, adds some functionality to it, and returns it. we can apply decorators using the @decorator_name syntax.

Below is an example of using decorator to record function execution time:

In [ ]:
# Define a simple decorator
from time import time

def my_decorator(func):
    def wrapper(): #wrapper is the function that allows us to modify the input function
        start_time = time()
        print(f"Execuation start time: {start_time}")
        func()
        end_time = time()
        print(f"Execuation end time: {end_time}")
        print(f"Duration: {end_time - start_time}")
    return wrapper

# Define a function and decorate it
@my_decorator
def say_hello():
    print("Hello!")

# Call the decorated function
say_hello()

Execuation start time: 1763602126.96466
Hello!
Execuation end time: 1763602126.964724
Duration: 6.413459777832031e-05
